# Farmer problem

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

In [1]:
using JuMP

In [2]:
using HiGHS

Commercial solvers as `CPLEX` or `Gurobi` can also be used.

# Average Scenario

We construct the model for the average scenario, using first the open-source HiGHS solver (https://highs.info/).

In [3]:
m = Model(HiGHS.Optimizer)

crops = ["Wheat", "Corn", "Sugar beets"]
ncrops = length(crops)

@variable(m, x[1:ncrops] >= 0)
@variable(m, y[1:2] >= 0)
@variable(m, w[1:4] >= 0)

costSeeding = [150, 230, 260]
costs = [238, 210]
prices = [170, 150, 36, 10]

4-element Vector{Int64}:
 170
 150
  36
  10

We can check that `m` is a JuMP model by checking its type.

In [4]:
typeof(m)

Model

We now add the constraints.

In [5]:
@constraint(m, surface, sum(x[i] for i=1:ncrops) <= 500)

surface : x[1] + x[2] + x[3] <= 500.0

In [6]:
@constraint(m, wheatNeeds, 2.5x[1]+y[1]-w[1] >= 200)

wheatNeeds : 2.5 x[1] + y[1] - w[1] >= 200.0

In [7]:
@constraint(m, cornNeeds, 3x[2]+y[2]-w[2] >= 240)

cornNeeds : 3 x[2] + y[2] - w[2] >= 240.0

In [8]:
@constraint(m, beetsProd, w[3]+w[4] <= 20x[3])

beetsProd : -20 x[3] + w[3] + w[4] <= 0.0

In [9]:
@constraint(m, beetsQuota, w[3] <= 6000)

beetsQuota : w[3] <= 6000.0

Let's define the objective function. 

In [10]:
@objective(m, Min, sum(costSeeding[i]*x[i] for i=1:ncrops) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))

150 x[1] + 230 x[2] + 260 x[3] + 238 y[1] + 210 y[2] - 170 w[1] - 150 w[2] - 36 w[3] - 10 w[4]

The complete model can be displayed with the command `print`.

In [11]:
print(m)

We now solve it.

In [12]:
optimize!(m)

Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
HiGHS run time      :          0.02


In [13]:
print("Solution: $(value.(x))")

Solution: [120.0, 80.0, 300.0]

In [14]:
? value

search: value values set_value fix_value has_values start_value callback_value



```
value(con_ref::ConstraintRef; result::Int = 1)
```

Return the primal value of constraint `con_ref` associated with result index `result` of the most-recent solution returned by the solver.

That is, if `con_ref` is the reference of a constraint `func`-in-`set`, it returns the value of `func` evaluated at the value of the variables (given by [`value(::VariableRef)`](@ref)).

Use [`has_values`](@ref) to check if a result exists before asking for values.

See also: [`result_count`](@ref).

## Note

For scalar constraints, the constant is moved to the `set` so it is not taken into account in the primal value of the constraint. For instance, the constraint `@constraint(model, 2x + 3y + 1 == 5)` is transformed into `2x + 3y`-in-`MOI.EqualTo(4)` so the value returned by this function is the evaluation of `2x + 3y`. ```

---

```
value(var_value::Function, con_ref::ConstraintRef)
```

Evaluate the primal value of the constraint `con_ref` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::VariableRef; result = 1)
```

Return the value of variable `v` associated with result index `result` of the most-recent returned by the solver.

Use [`has_values`](@ref) to check if a result exists before asking for values.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, v::VariableRef)
```

Evaluate the value of the variable `v` as `var_value(v)`.

---

```
value(var_value::Function, ex::GenericAffExpr)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericAffExpr; result::Int = 1)
```

Return the value of the `GenericAffExpr` `v` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, ex::GenericQuadExpr)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericQuadExpr; result::Int = 1)
```

Return the value of the `GenericQuadExpr` `v` associated with result index `result` of the most-recent solution returned by the solver.

Replaces `getvalue` for most use cases.

See also: [`result_count`](@ref).

---

```
value(p::NonlinearParameter)
```

Return the current value stored in the nonlinear parameter `p`.

# Example

```jldoctest; setup=:(using JuMP)
model = Model()
@NLparameter(model, p == 10)
value(p)

# output
10.0
```

---

```
value(ex::NonlinearExpression; result::Int = 1)
```

Return the value of the `NonlinearExpression` `ex` associated with result index `result` of the most-recent solution returned by the solver.

Replaces `getvalue` for most use cases.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, ex::NonlinearExpression)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(c::NonlinearConstraintRef; result::Int = 1)
```

Return the value of the `NonlinearConstraintRef` `c` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, c::NonlinearConstraintRef)
```

Evaluate `c` using `var_value(v)` as the value for each variable `v`.


In [15]:
methods(value)

# 17 methods for generic function "value":
[1] value(Q::LinearAlgebra.Symmetric{V, Matrix{V}}) where V<:AbstractVariableRef in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\sd.jl:346
[2] value(a::GenericAffExpr; result) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\aff_expr.jl:570
[3] value(::AbstractArray{<:AbstractJuMPScalar}) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1330
[4] value(var_value::Function, v::VariableRef) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1040
[5] value(var_value::Function, ex::GenericAffExpr{T, V}) where {T, V} in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\aff_expr.jl:330
[6] value(var_value::Function, ex::GenericQuadExpr{CoefType, VarType}) where {CoefType, VarType} in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\quad_expr.jl:676
[7] value(var_value::Function, ex::NonlinearExpression) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:363
[8] value(var_value::Function, c::NonlinearConstraintRef) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:520
[9] value(var_value::Function, con_ref::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex}) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\constraints.jl:913
[10] value(x::Number) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1338
[11] value(c::NonlinearConstraintRef; result) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:509
[12] value(con_ref::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex}; result) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\constraints.jl:900
[13] value(ex::GenericQuadExpr; result) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\quad_expr.jl:702
[14] value(::MutableArithmetics.Zero) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1337
[15] value(ex::NonlinearExpression; result) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:328
[16] value(p::NonlinearParameter) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:235
[17] value(v::VariableRef; result) in JuMP at C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1031

We will summarize the solution approach using a Julia function.

In [16]:
function farmer(factor::Float64 = 1.0, m::Model = Model(HiGHS.Optimizer))
    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m, x[1:ncrops] >= 0)
    @variable(m, y[1:2] >= 0)
    @variable(m, w[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = factor*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNeeds, returns[1]x[1]+y[1]-w[1] >= 200)
    @constraint(m, cornNeeds, returns[2]x[2]+y[2]-w[2] >= 240)
    @constraint(m, beetsProd, w[3]+w[4] <= returns[3]x[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min,sum(costSeeding[i]*x[i] for i=1:3) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)

    # println(value.(x))

    return m, status
end

farmer (generic function with 3 methods)

In [17]:
methods(farmer)

# 3 methods for generic function "farmer":
[1] farmer() in Main at In[16]:1
[2] farmer(factor::Float64) in Main at In[16]:1
[3] farmer(factor::Float64, m::Model) in Main at In[16]:1

In [18]:
maverage, status = farmer()

Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y, nothing)

In [19]:
maverage

A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y

In [20]:
print(maverage)

In [21]:
objective_value(maverage)

-118600.0

## Good scenario

In [22]:
mgood, status = farmer(1.2)

Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.6766666667e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.6766666667e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y, nothing)

In [23]:
mgood

A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y

In [24]:
objective_value(mgood)

-167666.6666666667

## Bad scenario

In [25]:
mbad, status = farmer(0.8)
#mbad, status = farmer(0.8, Model(solver = GurobiSolver()))

Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.6999813050e+02 Ph1: 2(2); Du: 2(469.998) 0s
          5    -5.9950000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     : -5.9950000000e+04
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y, nothing)

In [26]:
objective_value(mbad)

-59950.0

## Stochastic program - extensive form

We have to combine the three scenarios. We will assume that each one has a probability equal to 1/3.

In [ ]:
n = 3
p = [1/n for i = 1:n]

We will start with a naive implementation.

In [ ]:
function farmerStoch()
    m = Model(HiGHS.Optimizer)

    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m,x[1:ncrops] >= 0)

    @variable(m,ya[1:2] >= 0)
    @variable(m,wa[1:4] >= 0)

    @variable(m,yg[1:2] >= 0)
    @variable(m,wg[1:4] >= 0)

    @variable(m,yb[1:2] >= 0)
    @variable(m,wb[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returnsA = [2.5, 3, 20]
    returnsG = 1.2*[2.5, 3, 20]
    returnsB = 0.8*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNA, returnsA[1]x[1]+ya[1]-wa[1] >= 200)
    @constraint(m, cornNA, returnsA[2]x[2]+ya[2]-wa[2] >= 240)
    @constraint(m, beetsPA, wa[3]+wa[4] <= returnsA[3]x[3])
    @constraint(m, beetsQA, wa[3] <= 6000)

    @constraint(m, wheatNG, returnsG[1]x[1]+yg[1]-wg[1] >= 200)
    @constraint(m, cornNG, returnsG[2]x[2]+yg[2]-wg[2] >= 240)
    @constraint(m, beetsPG, wg[3]+wg[4] <= returnsG[3]x[3])
    @constraint(m, beetsQG, wg[3] <= 6000)

    @constraint(m, wheatNB, returnsB[1]x[1]+yb[1]-wb[1] >= 200)
    @constraint(m, cornNB, returnsB[2]x[2]+yb[2]-wb[2] >= 240)
    @constraint(m, beetsPB, wb[3]+wb[4] <= returnsB[3]x[3])
    @constraint(m, beetsQB, wb[3] <= 6000)

    @objective(m, Min, sum(costSeeding[i]*x[i] for i=1:3)
        + 1/3*sum(costs[i]*ya[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wa[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yg[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wg[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yb[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wb[i] for i=1:length(prices)))
    
    status = optimize!(m)

    return m, status
end

In [ ]:
mstoch, status = farmerStoch()

In [ ]:
mstoch

In [ ]:
valStoch = objective_value(mstoch)

## Expected value of the perfect information

In [ ]:
values = [objective_value(maverage), objective_value(mgood), objective_value(mbad)]

Let's compute the expected revenue under perfect information, simply obtained as the dot product between the probabilities vector and the optimal values vector.

In [ ]:
valPerfect = p'*values

In [ ]:
using LinearAlgebra

dot(p, values)

Expected value of perfect information:

In [ ]:
valPerfect - valStoch

## Value of the stochastic solution

We have to fix the first stage decision. We define the second stage problem.

In [ ]:
function secondStage(x, factor::Float64 = 1.0)
    m = Model(HiGHS.Optimizer)
    
    @variable(m,y[1:2] >= 0)   # purchases
    @variable(m,w[1:4] >= 0)   # sales

    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = (factor*[2.5, 3, 20]).*x
    z = [200 - returns[1], 240 - returns[2], returns[3]]
    
    @constraint(m, wheatNeeds, y[1]-w[1] >= z[1])
    @constraint(m, cornNeeds, y[2]-w[2] >= z[2])
    @constraint(m, beetsProd, w[3]+w[4] <= z[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min, sum(costs[i]*y[i] for i = 1:length(costs)) - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)
    
    return m, status
end

In [ ]:
x = [120, 80, 300]

In [ ]:
msecond, status = secondStage(x)

In [ ]:
msecond

In [ ]:
vaverage = objective_value(msecond)

In [ ]:
msecond, status = secondStage(x, 1.2)

In [ ]:
vgood = objective_value(msecond)

In [ ]:
msecond, status = secondStage(x, 0.8)

In [ ]:
vbad = objective_value(msecond)

Expected revenue

In [ ]:
er = costSeeding'*x+p'*[vaverage, vgood, vbad]

Value of the stochastic solution

In [ ]:
vss = valStoch - er